## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-11-16-54-25 +0000,nyt,Sarah Jessica Parker Describes Her Year as a B...,https://www.nytimes.com/2025/11/11/books/sarah...
1,2025-11-11-16-53-37 +0000,nypost,TikTok influencer and fraudster breaks silence...,https://nypost.com/2025/11/11/world-news/briti...
2,2025-11-11-16-52-13 +0000,nyt,How the Heavy-Metal Fall of a Dictator Shapes ...,https://www.nytimes.com/2025/11/11/us/politics...
3,2025-11-11-16-50-48 +0000,nyt,Pakistan ‘in a State of War’ After Explosion K...,https://www.nytimes.com/2025/11/11/world/asia/...
4,2025-11-11-16-47-44 +0000,nyt,Ban a Pro-Palestinian Group? The U.K. Governme...,https://www.nytimes.com/2025/11/11/world/europ...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2342/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
23,trump,43
133,shutdown,31
35,government,22
201,new,18
282,bbc,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
200,2025-11-11-01-13-00 +0000,wsj,Trump threatened to dock the pay of air-traffi...,https://www.wsj.com/politics/policy/trump-thre...,145
39,2025-11-11-15-24-00 +0000,wsj,Senate Minority Leader Chuck Schumer wasn’t on...,https://www.wsj.com/politics/policy/progressiv...,126
272,2025-11-10-21-11-40 +0000,nypost,Trump pledges to ‘abide’ by deal to end govern...,https://nypost.com/2025/11/10/us-news/trump-pl...,122
176,2025-11-11-03-12-00 +0000,wsj,The decision by eight members of the Senate De...,https://www.wsj.com/politics/policy/democrats-...,117
60,2025-11-11-14-02-00 +0000,wsj,The Republican-led Senate late Monday passed a...,https://www.wsj.com/politics/policy/senate-pas...,107


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
200,145,2025-11-11-01-13-00 +0000,wsj,Trump threatened to dock the pay of air-traffi...,https://www.wsj.com/politics/policy/trump-thre...
39,73,2025-11-11-15-24-00 +0000,wsj,Senate Minority Leader Chuck Schumer wasn’t on...,https://www.wsj.com/politics/policy/progressiv...
72,49,2025-11-11-13-25-07 +0000,nyt,Modi Vows Justice as Police Suspect Terrorism ...,https://www.nytimes.com/2025/11/11/world/asia/...
322,47,2025-11-10-17-49-00 +0000,wsj,The BBC reported that President Trump has thre...,https://www.wsj.com/politics/policy/bbc-chairm...
148,37,2025-11-11-08-43-05 +0000,wsj,Stock Market Today: S&P 500 Slips; ADP Data Po...,https://www.wsj.com/livecoverage/stock-market-...
242,35,2025-11-10-22-38-53 +0000,nyt,Families of Children Killed in Texas Floods Su...,https://www.nytimes.com/2025/11/10/us/camp-mys...
273,35,2025-11-10-21-02-09 +0000,wapo,Immigrant detention firm seeks immunity in cas...,https://www.washingtonpost.com/business/2025/1...
97,34,2025-11-11-11-33-19 +0000,nypost,Wisconsin teacher’s aide charged with having s...,https://nypost.com/2025/11/11/us-news/wisconsi...
95,33,2025-11-11-11-40-36 +0000,nypost,NYC gets first snow of season as powerful pola...,https://nypost.com/2025/11/11/us-news/polar-vo...
38,29,2025-11-11-15-25-48 +0000,nypost,"Christian ministry CEO, daughter ID’d as victi...",https://nypost.com/2025/11/11/us-news/christia...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
